In [1]:
import os, sys
sys.path.append("../../common/")
from python_tools import *

In [2]:
%matplotlib notebook

In [3]:
hdf5_filename = "/Users/wketchum/MicroBooNE_Data/MCC9/nue_optical_studies/nue_cosmic_opfilterana.hdf5"
root_filename = "/Users/wketchum/MicroBooNE_Data/MCC9/nue_optical_studies/nue_cosmic_opfilterana.root"
read_from_ROOT = False
write_to_hdf5 = False

if not os.path.isfile(hdf5_filename):
    read_from_ROOT = True
    write_to_hdf5 = True

In [4]:
t_df = pd.DataFrame()
p_df = pd.DataFrame()
opf_df = pd.DataFrame()

if read_from_ROOT:
    p_tree = uproot.open(root_filename)['nuana/particle_tree']
    t_tree = uproot.open(root_filename)['nuana/mctruth_tree']
    
    p_df = p_tree.pandas.df()
    p_df.set_index(["run","event","truth_index","p_index"],inplace=True)
#    p_df.drop_duplicates(subset=["run","event"],keep="first",inplace=True)
    
    t_df = t_tree.pandas.df()
#    t_df.drop_duplicates(subset=["run","event"],keep="first",inplace=True)
    t_df.set_index(["run","event","truth_index"],inplace=True)
    
    opf_tree = uproot.open(root_filename)['opfilterana/opfilter_anatree']
    opf_df = opf_tree.pandas.df()
#    opf_df.drop_duplicates(subset=["run","event"],keep="first",inplace=True)
    opf_df.set_index(["run","event"],inplace=True)
    
    if write_to_hdf5:
        t_df.to_hdf(hdf5_filename,"t_df")
        p_df.to_hdf(hdf5_filename,"p_df")
        opf_df.to_hdf(hdf5_filename,"opf_df")
else:
    t_df = pd.read_hdf(hdf5_filename,"t_df")
    p_df = pd.read_hdf(hdf5_filename,"p_df")
    opf_df = pd.read_hdf(hdf5_filename,"opf_df")

In [5]:
df_ev = t_df.copy()

p_df_nu = (p_df.query("status==0 and pdgcode==12").groupby(["run","event","truth_index"]).first()).copy()
p_df_e = (p_df.query("status==1 and pdgcode==11").groupby(["run","event","truth_index"]).first()).copy()
#p_df_nu.reset_index().drop_duplicates(subset=["run","event"],keep="first",inplace=True).set_index(["run","event","truth_index"],inplace=True)

df_ev = df_ev.merge(p_df_nu,how="inner",on=["run","event","truth_index"])
df_ev = df_ev.merge(p_df_e,how="inner",on=["run","event","truth_index"],suffixes=["","_e"])
df_ev = df_ev.merge(opf_df,how="inner",on=["run","event"])


In [6]:
df_ev["ke_e"] = df_ev["e_e"]-df_ev["mass_e"]
df_ev[["e","e_e","mass_e","ke_e"]]

e       e_e   mass_e      ke_e
run event                                       
1   1      0.817987  0.583827  0.00051  0.583317
    3      1.066837  0.783178  0.00051  0.782668
    4      0.647949  0.430851  0.00051  0.430341
    5      0.424623  0.351221  0.00051  0.350711
    6      0.197563  0.135801  0.00051  0.135291
    7      0.197563  0.082786  0.00051  0.082276
    8      1.066837  0.771237  0.00051  0.770727
    9      1.214426  0.991630  0.00051  0.991120
    10     0.570512  0.313128  0.00051  0.312618
    11     0.197563  0.091705  0.00051  0.091195
    12     2.427924  0.296555  0.00051  0.296045
    13     2.427924  0.725058  0.00051  0.724548
    14     1.156822  0.692236  0.00051  0.691726
    15     0.713320  0.608488  0.00051  0.607978
    16     1.653085  1.528255  0.00051  1.527745
    17     2.427924  1.150486  0.00051  1.149976
    18     1.285633  0.180664  0.00051  0.180154
    19     1.156822  0.623685  0.00051  0.623175
    20     2.760877  1.846744  0.00051  1.846234
    20001  1.574802  1.426861  0.00051  1.426351
    20002  1.237014  0.407716  0.00051  0.407206
    20003  1.282120  0.833168  0.00051  0.832658
    20004  0.515692  0.307282  0.00051  0.306772
    20005  1.451910  0.657952  0.00051  0.657442
    20006  1.710869  0.279731  0.00051  0.279221
    20007  1.868474  1.756706  0.00051  1.756196
    20008  0.864186  0.309876  0.00051  0.309366
    20009  1.474771  1.329744  0.00051  1.329234
    20010  2.026769  0.783585  0.00051  0.783075
    20012  0.666040  0.615738  0.00051  0.615228
...             ...       ...      ...       ...
    1991   0.918121  0.726138  0.00051  0.725628
    1992   0.918121  0.564456  0.00051  0.563946
    1993   0.823715  0.245910  0.00051  0.245400
    1994   3.177546  1.468078  0.00051  1.467568
    1995   0.918121  0.761467  0.00051  0.760957
    1996   1.818835  0.974158  0.00051  0.973648
    1997   1.939350  1.752263  0.00051  1.751753
    1998   1.053252  0.154799  0.00051  0.154289
    1999   0.683954  0.476606  0.00051  0.476096
    2000   0.777434  0.387987  0.00051  0.387477
    181    0.841427  0.234424  0.00051  0.233914
    182    0.259840  0.075287  0.00051  0.074777
    183    0.628130  0.100329  0.00051  0.099819
    184    0.531325  0.449150  0.00051  0.448640
    185    1.284872  0.203932  0.00051  0.203422
    186    0.807997  0.172096  0.00051  0.171586
    187    1.088113  1.006746  0.00051  1.006236
    188    2.279377  0.475315  0.00051  0.474805
    189    2.765618  1.197726  0.00051  1.197216
    190    1.437600  1.067401  0.00051  1.066891
    191    1.062838  0.931194  0.00051  0.930684
    192    1.284872  0.376158  0.00051  0.375648
    193    2.279377  1.957760  0.00051  1.957250
    194    0.841427  0.480243  0.00051  0.479733
    195    0.807997  0.551285  0.00051  0.550775
    196    1.982965  1.174169  0.00051  1.173659
    197    1.982965  1.522657  0.00051  1.522147
    198    1.553768  1.085929  0.00051  1.085419
    199    2.312177  0.836325  0.00051  0.835815
    200    0.962283  0.817175  0.00051  0.816665

[129580 rows x 4 columns]

In [7]:
tolerance = 10.0
min_x = np.min(df_ev["start_x"])+tolerance
max_x = np.max(df_ev["start_x"])-tolerance
min_y = np.min(df_ev["start_y"])+tolerance
max_y = np.max(df_ev["start_y"])-tolerance
min_z = np.min(df_ev["start_z"])+tolerance
max_z = np.max(df_ev["start_z"])-tolerance

In [8]:
my_df=df_ev.query("start_x>%f and start_x<%f and start_y>%f and start_y<%f and start_z>%f and start_z<%f"%(min_x,max_x,min_y,max_y,min_z,max_z))

In [9]:
np.mean(df_ev["pe_beam"])

3694.7878

In [ ]:
#my_df=df_ev.query("start_x>min_x+10.0 and start_x<max_x-10.0 and start_y>min_y+10.0 and start_y<max_y-10.0 and start_z>min_z+10.0 and start_z<max_z-10.0")

fig,axes = plt.subplots()

axes.hist2d(my_df["e"],my_df["pe_beam"],bins=[50,75],range=[[0.,2.],[0,2000]])
plt.title("BNB Intrinsic $\\nu_e$ CC (10cm fiducial)")
axes.set_xlabel("$E_\\nu$ (GeV)")
axes.set_ylabel("Beam PHMax (PE?)")
plt.show()
#plt.savefig("plots/genie_1mu1p_costheta_v_enu.pdf")

In [ ]:
fig,axes = plt.subplots()

axes.hist2d(my_df["e"],my_df["pe_beam"],bins=[50,40],range=[[0.,2.],[0,500]])
plt.title("BNB Intrinsic $\\nu_e$ CC (10cm fiducial), Zoomed")
axes.set_xlabel("$E_\\nu$ (GeV)")
axes.set_ylabel("Beam PHMax (PE?)")
plt.show()
#plt.savefig("plots/genie_1mu1p_costheta_v_enu.pdf")

In [ ]:
fig,axes = plt.subplots()

bins_pe_beam = np.linspace(0.0,10000,500)
plt.grid()
plt.title("BNB Intrinsic $\\nu_e$ CC (10cm fiducial), Optical Filter Efficiency")
axes.hist(my_df["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='r',label='All $E_\\nu$',**pltops_hist)
axes.hist(my_df.query("e<1.0")["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='b',label='$E_\\nu<1$ GeV',**pltops_hist)
axes.hist(my_df.query("e<0.5")["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='g',label='$E_\\nu<500$ MeV',**pltops_hist)
axes.hist(my_df.query("e<0.2")["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='orange',label='$E_\\nu<200$ MeV',**pltops_hist)
axes.set_xlim(0.0,5000)
axes.set_ylabel("Pass Efficiency")
axes.set_xlabel("Beam PHMax, Minimum Threhsold (PE)")
plt.legend()
plt.show()
#plt.savefig("plots/genie_1mu1p_costheta_v_enu.pdf")

In [ ]:
fig,axes = plt.subplots()

bins_pe_beam = np.linspace(0.0,10000,5000)
plt.grid()
plt.title("BNB Intrinsic $\\nu_e$ CC (10cm fiducial), Optical Filter Efficiency")
axes.hist(my_df["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='r',label='All $E_\\nu$',**pltops_hist)
axes.hist(my_df.query("e<1.0")["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='b',label='$E_\\nu<1$ GeV',**pltops_hist)
axes.hist(my_df.query("e<0.5")["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='g',label='$E_\\nu<500$ MeV',**pltops_hist)
axes.hist(my_df.query("e<0.2")["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='orange',label='$E_\\nu<200$ MeV',**pltops_hist)
axes.set_xlim(0.0,500)
axes.set_ylim(0.5,1.05)
axes.set_ylabel("Pass Efficiency")
axes.set_xlabel("Beam PHMax, Minimum Threhsold (PE)")
plt.legend(loc=3)
plt.show()
#plt.savefig("plots/nue_opfilter_eff_zoom.pdf")

In [ ]:
fig,axes = plt.subplots()

bins_pe_beam = np.linspace(0.0,10000,5000)
plt.grid()
plt.title("BNB Intrinsic $\\nu_e$ CC (10cm fiducial), Optical Filter Efficiency, $E_\\nu>200$ MeV")
axes.hist(my_df.query("e>0.2")["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='r',**pltops_hist)
axes.set_xlim(0.0,100)
#axes.set_ylim(0.9,1.05)
axes.set_ylabel("Pass Efficiency")
axes.set_xlabel("Beam PHMax, Minimum Threhsold (PE)")
#plt.legend(loc=3)
plt.show()
#plt.savefig("plots/nue_opfilter_eff_e_gt_200MeV.pdf")

In [ ]:
total_x_span = max_x-min_x
x_q1_lo = min_x
x_q1_hi = x_q1_lo+total_x_span*0.25
x_q2_lo = x_q1_hi
x_q2_hi = x_q2_lo+total_x_span*0.25
x_q3_lo = x_q2_hi
x_q3_hi = x_q3_lo+total_x_span*0.25
x_q4_lo = x_q3_hi
x_q4_hi = x_q4_lo+total_x_span*0.25

fig,axes = plt.subplots()

bins_pe_beam = np.linspace(0.0,50000,5000)
plt.grid()
plt.title("BNB Intrinsic $\\nu_e$ CC (10cm fiducial), Optical Filter Efficiency, $E_\\nu<500$ MeV")
axes.hist(my_df.query("e<0.5 and start_x>%f and start_x<%f"%(x_q1_lo,x_q1_hi))["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='r',label='$%2.1f cm < x < %2.1f cm$'%(x_q1_lo,x_q1_hi),**pltops_hist)
axes.hist(my_df.query("e<0.5 and start_x>%f and start_x<%f"%(x_q2_lo,x_q2_hi))["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='b',label='$%2.1f cm < x < %2.1f cm$'%(x_q2_lo,x_q2_hi),**pltops_hist)
axes.hist(my_df.query("e<0.5 and start_x>%f and start_x<%f"%(x_q3_lo,x_q3_hi))["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='g',label='$%2.1f cm < x < %2.1f cm$'%(x_q3_lo,x_q3_hi),**pltops_hist)
axes.hist(my_df.query("e<0.5 and start_x>%f and start_x<%f"%(x_q4_lo,x_q4_hi))["pe_beam"],bins=bins_pe_beam,cumulative=-1,density=True,color='orange',label='$%2.1f cm < x < %2.1f cm$'%(x_q4_lo,x_q4_hi),**pltops_hist)
axes.set_xlim(0.0,500)
axes.set_ylim(0.5,1.05)
axes.set_ylabel("Pass Efficiency")
axes.set_xlabel("Beam PHMax, Minimum Threhsold (PE)")
plt.legend(loc=3)
plt.show()
plt.savefig("plots/nue_opfilter_eff_xlines.pdf")

In [ ]:
df_ev.query("pe_beam<10")[["e","start_x","start_y","start_z","start_t"]]

In [ ]:
print np.min(df_ev["start_x"])
print np.max(df_ev["start_x"])
print np.min(df_ev["start_y"])
print np.max(df_ev["start_y"])
print np.max(df_ev["start_z"])
print np.min(df_ev["start_z"])

In [ ]:
bwidthx = hden[1][1]-hden[1][0]
bwidthy = hden[2][1]-hden[2][0]

In [ ]:
fig,axes = plt.subplots(2)

hden, xedge, yedge, im = axes[0].hist2d(my_df["start_x"],my_df["ke_e"],bins=[50,50],range=[[0.,250.],[0,1.0]])
hnum, xedge, yedge, im  = axes[1].hist2d(my_df.query("pe_beam>2000.0")["start_x"],my_df.query("pe_beam>2000.0")["ke_e"],bins=[50,50],range=[[0.,250.],[0,1.0]])
plt.show()


In [ ]:
fig,axes = plt.subplots()

hden, xedge, yedge, im = axes.hist2d(my_df["start_x"],my_df["ke_e"],bins=[50,50],range=[[0.,250.],[0,1.0]])
hnum, xedge, yedge, im  = axes.hist2d(my_df.query("pe_beam>20.0")["start_x"],my_df.query("pe_beam>20.0")["ke_e"],bins=[50,50],range=[[0.,250.],[0,1.0]])

heff = np.nan_to_num(hnum/hden)
heff = np.transpose(heff)
bwidthx = xedge[1]-xedge[0]
bwidthy = yedge[1]-yedge[0]
extent = (xedge[0]-0.5*bwidthx, xedge[-1]-0.5*bwidthx, (yedge[0]-0.5*bwidthy)*1000., (yedge[-1]-0.5*bwidthy)*1000.)
plt.clf()
plt.imshow(heff, origin="lower",extent=extent,aspect="auto")

plt.colorbar(label='Efficiency')
plt.xlabel('Distance from anode (cm)')
plt.ylabel('$KE_e$ (MeV)')
plt.show()
plt.savefig("plots/nue_opfilter_eff_electron_ke_vs_x.pdf")

In [ ]:
fig,axes = plt.subplots()

hden, xedge, yedge, im = axes.hist2d(my_df["start_x"],my_df["ke_e"],bins=[50,50],range=[[0.,250.],[0,0.2]])
hnum, xedge, yedge, im  = axes.hist2d(my_df.query("pe_beam>20.0")["start_x"],my_df.query("pe_beam>20.0")["ke_e"],bins=[50,50],range=[[0.,250.],[0,0.2]])

heff = np.nan_to_num(hnum/hden)
heff = np.transpose(heff)
bwidthx = xedge[1]-xedge[0]
bwidthy = yedge[1]-yedge[0]
extent = (xedge[0]-0.5*bwidthx, xedge[-1]-0.5*bwidthx, (yedge[0]-0.5*bwidthy)*1000., (yedge[-1]-0.5*bwidthy)*1000.)
plt.clf()
plt.imshow(heff, origin="lower",extent=extent,aspect="auto")

plt.colorbar(label='Efficiency')
plt.xlabel('Distance from anode (cm)')
plt.ylabel('$KE_e$ (MeV)')
plt.show()
plt.savefig("plots/nue_opfilter_eff_electron_ke_vs_x_zoom.pdf")

In [ ]:
print heff

In [ ]:
xedge[0]

In [ ]:
extent=(hden[1][0],hden[1][-1],hden[2][0],hden[2][-1])
print extent

In [ ]:
print heff.shape
print hden[1].size-1

In [ ]:
xarr=np.array([])
yarr=np.array([])
zarr=np.array([])
for ix in range(hden[1].size-1):
    x=0.5*(hden[1][ix+1]-hden[1][ix])+hden[1][ix]
    for iy in range(hden[2].size-1):
        y=0.5*(hden[2][iy+1]-hden[2][iy])+hden[2][iy]
        xarr=np.append(xarr,x)
        yarr=np.append(yarr,y)
        zarr=np.append(zarr,hden[0][ix][iy])
arr, yarr = np.meshgrid(xarr, yarr)

In [ ]:


from mpl_toolkits.mplot3d import Axes3D # This import has side effects required for the kwarg projection='3d' in the call to fig.add_subplot
from matplotlib import cm

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.plot_surface(hden[1][0:50],hden[2][0:50],heff,rstride=1, cmap=cm.jet, cstride=1,
        linewidth=0, antialiased=False)
plt.show()